In [1]:
%reload_ext autoreload
%autoreload 2

## 内容约定

- _所有提示错误,需要带上该层的名字, 如:Conv2d_abcdef_

- ~~所有层的shape最后要把第一维度设置为batch_size的值, 如(3, 28, 28)->(32, 3, 28, 28), 其中32为batch_size的值~~

- _当参数配置修改了batch_size, input_size时, 拖拽的模型需要重新计算更新shape值_

## 前端任务

**功能/其他**

1. [x] 根据参数`batch_size, input_size, num_class`属性, 对相关层的参数做默认配置(如:Input层和Output层)


**删除/隐藏**

1. [ ] <font color='Green'><strong>有修改</strong></font> [删除Loss层 ~~和 BaseFunc层~~](#del_loss)
- [ ] <font color='Blue'><strong>有修改</strong></font> [隐藏或者注释掉Conv层的**ConvTransposexx**所有子类型](#del_devconv)
- [ ] [隐藏或者注释掉Vulkan层的Reshape子类型](#del_reshape)
- [ ] **有修改** [删除Pooling层属性中的return_indices和**dilation**](#del_pool_attrs)
- [x] [删除Normalize层属性中的eps](#del_norm_attrs)
- [x] [只读处理Vukan子类Flatten层属性中的start_dim和end_dim](#del_flatten_attrs)


**错误提示/形状计算**

为了减少错误提示, 前端可以封装属性控件, 如: int控件,float控件, 每个控件有值域范围, 保证用户输入的值不会超出范围.

1. [x] [Input](#input_layer)
- [x] [Output](#output_layer)
- [ ] [Convolution](#conv_layer)
- [ ] <font color='Green'><strong>新添加</strong></font> [BaseFunc.Add](#add_layer)
- [ ] <font color='Green'><strong>新添加</strong></font> [BaseFunc.Cat](#cat_layer)
- [ ] **新添加** [Padding](#padding_layer)
- [ ] **新添加** [Pooling](#pooling_layer)
- [ ] **新添加** [Activation](#act_layer)
- [ ] **新添加** [Normalize](#norm_layer)
- [ ] **新添加** [Vulkan.Flatten](#flatten_layer)
- [ ] **新添加** [Linear](#linear_layer)
- [ ] **新添加** [Dropout](#dropout_layer)

除了参考该文档, 还可以查看原go-js版本实现[layernode.js](/edit/k12libs/www/js/cauchy/layernode.js)


**体验/问题单**

1. [ ] 属性设置后直接点击下一步, 属性值实际上没有改变. (属性设置触发事件问题)
2. [ ] BaseFun.Add层给的参数不对

<span id="del_loss"/>

### 删除Loss层

1. Loss的配置不需要放到模型拖拽画布中, 直接放到参数拖拽配置里

![](/notebooks/assets/layers/delete-1.png)

<span id="del_devconv"/>

### 隐藏或者注释掉Conv层的ConvTranspose2d子类型

对于简单的分类模型, 暂时用不到反卷积(ConvTranspose2d), 但后续如果添加生成对抗网络可能需要

![](/notebooks/assets/layers/delete-2.png)


<span id="del_reshape"/>

### 隐藏或者注释掉Vulkan层的Reshape子类型

直接使用flatten代替

![](/notebooks/assets/layers/delete-6.png)


<span id="del_pool_attrs"/>

### 删除Pooling层属性中的return_indices和dilation

![](/notebooks/assets/layers/delete-3.png)


<span id="del_norm_attrs"/>

### 删除Normalize层属性中的eps

eps只是为了防止除零操作, 对用户没有学习意义, 暂做删除操作

![](/notebooks/assets/layers/delete-4.png)

<span id="del_flatten_attrs"/>

### 只读处理Vukan子类Flatten层属性中的start_dim和end_dim 

flatten目的就是要把多维数据拉伸到一维空间, start_dim和end_dim是必要属性但是不允许修改. (只读处理)

![](/notebooks/assets/layers/delete-5.png)

<span id="input_layer"/>

### Input层

作为模型的第一个层(必须存在, 否则提示"**模型非法,缺少输入层**"), 属性in_features完全由参数拖拽配置中的input_size决定, 同样该层的shape大小也是固定的(batch_size, 3, input_size.width, input_size.height), 如下:
![](/notebooks/assets/layers/input_shape.png)

上图对应的参数配置(**@冬至**):

> ```json
> {
>     "train.batch_size": 32,
>     "train.data_transformer.input_size": [28, 28]
> }
> ```

-----------

#### 参数
    
属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:----
<img width=150/> | <img width=100/> | <img width=200/> | <img width=450/>
in_features | `[int, int, int]` | `[3, width, height]` | 参数必须包含三个值; 非法时提示"in_features参数错误" <br>[0]: 通道数,一般固定位3; 非法时提示"in_features参数错误" <br> [1]: 用户设置的图片input_size的width; 非法时提示"in_features参数错误"<br> [2]: 用户设置的图片input_size的height; 非法时提示"in_features参数错误"


Input层`in_features`可以设置成**只读**, 其值为`(3, input_size.width, input_size.height)`, 从而减少用户输入错误.


------------


#### 形状

输入形状: 无

输出形状: (B, C, W, H) // (batch_size:批量大小, channel:通道数3, width:输入宽度, height:输出宽度) 

计算方式:

> ```js
> shape = {b: x, c: this.in_features[0], w: this.in_features[1], h: this.in_features[2]}
> ```

------------

### 实例

![Input Node](/notebooks/assets/layers/input_layer.png)

<span id="output_layer"/>

### Output层

作为模型拖拽的最后一层, 只有一个输入层(一个箭头指向它), 不允许有多个箭头指向该层, 否则提示"**输出层连线错误!**", 属性out_features的值必须等于(pre)输入层shape的最后一维度的值并且等于分类个数(num_classes), out_features可以做成不允许修改(**只读**), 如下图:

![](/notebooks/assets/layers/output_classes.png)

上图对应的参数配置(**@冬至**):

> ```json
> {
>     "data.num_classes": 10
> }
> ```

----------

#### 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:----
<img width=150/> | <img width=100/> | <img width=200/> | <img width=450/>
out_features | int | $prelayer.shape.h$ | 其值应该等于分类的数目, 并等于前一个层输入shape的最后一维度的值 (实际上这个层没有意义, 只用来作为显示)

----------

#### 形状

输入形状: 任意

输出形状: 等于输入形状

形状计算:

> ```js
>     let b = prelayer.shape.b; // 前一层(输入层)shape输出的批量大小(batch_size)
>     let c = prelayer.shape.c; // 前一层(输入层)shape输出的通道(channel), 可能没有为空
>     let w = prelayer.shape.w; // 前一层(输入层)shape输出的宽(width), 可能没有为空
>     let h = prelayer.shape.h; // 前一层(输入层)shape输出的高(height), 必须有, 表示shape维度的最后一维度
>     shape = {b: b, c:c, w:w, h:h}
> ```

----------------

#### 实例

![Output Node](/notebooks/assets/layers/output_layer.png)

<span id="conv_layer"/>


### Convolution层 (2D)

卷积的属性值域合法值比较难判断, 为了简单某些值域的使用$\infty$表示, 从而简化一些判断逻辑. **padding的值域除外**

-------------

#### 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=200/> | <img width=450/>
in_channels | int | $prelayer.shape.c$ | 非法时提示"参数in_channels应为大于0的整数, 且值为上一层的shape输出中的shape.c" <br> 为了防止用户输入错误,可以改为只读形式,值为前一层的shape中channel
out_channels |  int | $[1, +\infty)$ | 非法时提示"参数out_channels应为大于0的整数"
kernel_size | int | $[3, +\infty)$ | 非法时提示"参数kernel_size应为大于0的整数 (推荐3,5,7等最大公约数为1的数)"
stride | int | $[1, +\infty)$ | 非法时提示"参数stride应为大于0的整数"
padding | int | $[0, x]$ | 非法时提示"参数padding应为大于等于0的整数"
dilation | int | $[1, \infty)$ | 非法时提示"参数dilation应为大于0的整数"
groups | int | $[1, \infty)$ | 非法时提示"参数groups为大于0的整数"
bias | bool | True/False |


**padding不应大于`x = {kernel_size + (kernel_size - 1)*(dilation - 1)} / 2`**否则提示"**参数padding值非法**"

-------

#### 形状

输入形状: (B, $C_{in}$, W, H)

输出形状: (B, $C_{out}$, $W_{out}$, $H_{out}$)

形状计算:

> ```js
>     let b = prelayer.shape.b; // 前一层shape输出的批量大小(batch_size)
>     let w = prelayer.shape.w; // 前一层shape输出的宽(width)
>     let h = prelayer.shape.h; // 前一层shape输出的高(height)
>     //考虑到dilation大于1的情况
>     let k = eval(`${this.kernel_size} + (${this.kernel_size} - 1) * (${this.dilation} - 1)`); 
>
>     wout = Math.floor(eval(`(${w} + 2 * ${this.padding} - ${k})/${this.stride}`)) + 1;
>     hout = Math.floor(eval(`(${h} + 2 * ${this.padding} - ${k})/${this.stride}`)) + 1;
>     
>     shape = {b: b, c: this.out_channels, w: wout, h: hout}
> ```

-------

#### 实例

当dilation == 1时: `k = kernel_size`

![Convoluation Node](/notebooks/assets/layers/convolution_layer.png)

当dilation > 1时, `k = kernel_size + (kernel_size -1)*(dilation -1)`

$$
\begin{align*}
shape.w &= (preinput.w + 2*padding - k) / stride + 1 \\
shape.h &= (preinput.h + 2*padding - k) / stride + 1
\end{align*}
$$


<span id="padding_layer"/>

### Padding层(只考虑中心对称)

`零填充, 常数填充, 镜像填充, 重复填充`几种子类型方式可按一种方式处理, 只考虑对称方式的填充(top = bottom = left = right)

---------------

#### 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=200/> | <img width=450/>
padding | int | $[0, +\infty]$ | 意义为(padding_left, padding_right, padding_top, padding_bottim)
value | float | $[0, +\infty]$ | 常数填充时, 需要填充的值

--------------

#### 形状

输入形状: (B, C, W, H)

输出形状: (B, C, $W_{out}$, $H_{out}$)

形状计算:

> ```js
>     let b = prelayer.shape.b;
>     let w = prelayer.shape.w + 2*padding
>     let h = prelayer.shape.h + 2*padding
>     let c = prelayer.shape.c
>     
>     shape = {b:b, c: c, w: w, h: h}
> ```

--------------

#### 实例

![Padding Node](/notebooks/assets/layers/padding_layer.png)

<span id="pooling_layer"/>

### Pooling层

#### 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=200/> | <img width=450/>
kernel_size | int | $[3, +\infty]$ | 非法时提示"参数kernel_size应为大于0的整数 (推荐3,5,7等最大公约数为1的数)"
stride | int | $[1, +\infty]$ | 非法时提示"参数stride应为大于0的整数"
padding | int | $[0, +\infty]$ | 非法时提示"参数padding应为大于等于0的整数"
output_size | int | $[1, +\infty]$ | (only AdaptiveXXXPool)自动适配池化的输出大小, 非法时提示"参数output_size应为大于0的整数"
ceil_mode | bool | True/False | True: 向上取整; False: 向下取整
~~dilation~~ | ~~int~~ | ~~$[1, +\infty]$~~ | ~~非法时提示"参数dilation应为大于0的整数"~~
~~return_indices~~ | ~~bool~~ | ~~True/False~~ | ~~(only MaxPool)记录池化像素索引, 用来反池化, **可以先忽略/隐藏该参数**~~
~~count_include_pad~~ | ~~bool~~ | ~~True/False~~ | ~~(only AvgPool)填充值是否参与计算, **可以先忽略/隐藏该参数**~~

**output_size参数目前只需考虑是W==H的情况, W!=H暂不考虑, 所以output_size是个整型即可, W=H=output_size**

--------------

#### 形状

输入形状: (B, C, W, H)

输出形状: (B, C, $W_{out}$, $H_{out}$)

形状计算:

> ```js
>     let b = prelayer.shape.b;   // 前一层shape的批量
>     let c = prelayer.shape.c;   // 前一层shape的通道
>     let w = prelayer.shape.w;   // 前一层shape的宽度
>     let h = prelayer.shape.h;   // 前一层shape的高度
>  
>     if (subtype.startsWith('Adaptive')) {
>         let W = output_size;
>         let H = output_size;
>     } else {
>         if (ceil_mode == "True") {
>             func = Math.ceil;     // 上取整方法
>         } else { 
>             func = Match.floor;   // 下取整方法
>         }
> 
>         // 删除dilation let K = eval(`${kernel_size} + (${kernel_size} - 1) * (${dilation} - 1)`);
>         let W = func(eval(`(${w} + 2 * ${padding} - ${kernel_size})/${stride}`)) + 1;
>         let H = func(eval(`(${h} + 2 * ${padding} - ${kernel_size})/${stride}`)) + 1;
>     }
>     shape = {b: b, c: c, w: W, h: H};
> ```

--------------

#### 实例

![Pooling Node](/notebooks/assets/layers/pooling_layer.png)

<span id="act_layer"/>

### Activation层

该层不会对shape做变化

---------

#### 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=200/> | <img width=450/>
inplace | bool | True/False

---------

#### 形状

输入形状: 任意

输出形状: 任意

形状计算:

> ```js    
>     let b = prelayer.shape.b;
>     let c = prelayer.shape.c;
>     let w = prelayer.shape.w;
>     let h = prelayer.shape.h;
>     
>     shape = {b: b, c: c, w: w, h: h}
> ```

---------

#### 实例

![Activation Node](/notebooks/assets/layers/activation_layer.png)

<span id="norm_layer"/>

### Normalize层

$$
y = \frac{x - \mathrm{E}[x]}{ \sqrt{\mathrm{Var}[x] + \epsilon}} * \gamma + \beta
$$

------------------

#### 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=200/> | <img width=450/>
num_features | int | prelayer.shape.c | 与上一层的shape.c相等, 若非法提示"num_features特征数目不对"; **推荐设置成只读**
~~eps~~ | float | $(0, 1)$ | 防止分母为0, 所以该属性**推荐隐藏** 
momentum | float | $(0, 1)$ | 动态均值和动态方差所使用的动量因子(新batch和全局的取舍$\hat{x}_\text{new} = (1 - \text{momentum}) \times \hat{x} + \text{momentum} \times x_t$)
affine | bool | True/False | True: 对结果缩放($\gamma$)和平移($\beta$), 均可在反向传播中学习
track_running_stats | bool | Ture/False | True: 启动全局batch统计特性

**所有不在值域范围内的参数值均可提示参数非法**

------------------

#### 形状

输入形状: (B, C, W, H)

输出形状: (B, C, W, H)

形状计算:

> ```js
>     let b = prelayer.shape.b
>     let c = prelayer.shape.c
>     let w = prelayer.shape.w
>     let h = prelayer.shape.h 
>     
>     shape = {b: b, c: c, w: w, h: h}
> ```

------------------

#### 实例

![Normal Node](/notebooks/assets/layers/normalize_layer.png)

<span id="flatten_layer"/>

### Flatten层

将多维数据"压平"为1维

----------

#### 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=200/> | <img width=450/>
~~start_dim~~ | int | $[0, shape.length - 1]$ | **默认值, 只读属性**
~~end_dim~~ | int | $[-1, shape.length - 1]$ | **默认值, 只读属性**

-----------

### 形状

输入形状: (B, C, W, H)

输出形状: (B, C*W*H)

形状计算:

> ```js
>     let b = prelayer.shape.b;
>     let c = prelayer.shape.c;
>     let w = prelayer.shape.w;
>     let h = prelayer.shape.h;
>     
>     shape = {b: b, c: null, c: null, h: w*h*c}
> ```

----------

#### 实例

![Flatten Node](/notebooks/assets/layers/flatten_layer.png)

### ~~Reshape层~~

**使用flatten层取代**

---------------

#### 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=100/> | <img width=450/>
target_shape | list(int) | len(list) <= 3 | list里所有元素的乘积应该等于上一层shape的w*h*c <br> 非法提示"target_shape参数非法" 
    
---------------

##### 形状
    
```js
    let b = prelayer.shape.b;
    let c = prelayer.shape.c;
    let w = prelayer.shape.w;
    let h = prelayer.shape.h;

    let C = null;
    let W = null;
    let H = null;
    switch(len(target_shape)) {
        case 3:
            C = target_shape[0];
            W = target_shape[1];
            H = target_shape[2];
            break;
        case 2:
            W = target_shape[0];
            H = target_shape[1];
            break;
        case 1:
            H = target_shape[0];
            break;
        case *:
            // error
    }

    shape = {b: b, c: C, w: W, h: H}

```

---------------

#### 实例

![Reshape Node](/notebooks/assets/layers/reshape_layer.png)

<span id="linear_layer"/>

### Linear层

全连接 

-----------

#### 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=200/> | <img width=450/>
in_features | int | prelayer.shape.h | 值必须为上个层的shape.h, 非法提示"in_features参数非法"; **可以设置成只读**
out_features | int | $[1, +\infty]$ |
bias | bool | True/False |

-----------

#### 形状

输入形状: (B, H)

输出形状: (B, $H_{out}$)

计算形状:

> ```js
>     let b = prelayer.shape.b;
>     let h = out_features;
>     shape = {b: b, c: null, w:null, h:h}
> ```

-----------

#### 实例

![Linear Node](/notebooks/assets/layers/linear_layer.png)

<span id="dropout_layer"/>

### Dropout层

----------

#### 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=100/> | <img width=450/>
p | float | $(0, 1)$ | 伯努利(0-1)以p几率让神经元失效
inplace | bool | True/False | True: 对输入tensor本地修改, 节省内容 **推荐为默认**

----------

#### 形状

输入形状: 任意

输出形状: 等于输入

----------

<span id="add_layer"/>

### Add层

该层无参数, 只需保证两个被加层的shape相同, 如果不一致, 需要提示错误"**两个输入层的形状不一致**"

----------

#### 形状

输入形状: 两个任意(两个input层维度一致,大小一致)

输出形状: 输入形状

-----------

#### 实例

![](/notebooks/assets/layers/add_layer.png)

<span id="cat_layer"/>

### Cat层

输入层至少2个, 且它们的shape的维度大小`shape.length`要相同, 如: `pre1.shape.length = pre2.shape.length = pre3.shape.length`, 除了要合并的维度上的值不同,其他维度对应的值必须一致, 如: 在dim=1维度合并 `pre1.shape = (32, x, 28, 28); pre2.shape = (32, y, 28, 28)`

-----------

#### 参数

属性 | 类型 | 值域 | 描述
:---:|:---:|:---:|:------
<img width=150/> | <img width=100/> | <img width=200/> | <img width=450/>
dim | int | $[0, prelayer.shape.length)$ | -1: 表示最后一维度, 输入的层要大于2个, 除了要合并的dim外, 其他维度必须一致, 否则提示"dim参数错误"

-----------

#### 形状

输入形状: 多个形如$(B, C, W, H)$, 目标维度值可以不一样, 其他维度值一致

输出形状: $(B', C', W', H')$

形状计算:

> ```js
>     //            0  1   2   3
>     // dim= 1:  (32, 3, 64, 64), (32, 3, 64, 64) --> (32, 3+3, 64, 64)
>     // dim=-1:  (32, 3, 64, 64), (32, 3, 64, 64) --> (32, 3, 64, 64 + 64)
>     let num_layer = prelayers.length;
>     let len_shape = prelayers[0].shape.length;
>     
>     let shapes = [];
>     for(let i = 0; i < num_layer; ++i){
>         let b = parseInt(prelayers[i].shape.b);
>         let c = parseInt(prelayers[i].shape.c);
>         let w = parseInt(prelayers[i].shape.w);
>         let h = parseInt(prelayers[i].shape.h);
>         shapes.push([b, c, w, h]);
>     }
>     
>     if (dim == -1) { // 将 -1 转为shape的最后维度索引
>         dim = len_shape - 1
>     }
>     
>     for (let i = 1; i < num_layer; ++i) { // 遍历剩余层
>         for (let j = 0; j < len_shape; ++j) { // 遍历每层shape的维度
>             if (j == dim) { // 合并维度
>                 shapes[0][j] = shapes[0][j] + shapes[i][j] 
>             } else { // 其他维度必须保持一致
>                 if (shapes[0][j] != shapes[i][j]) {
>                     // 提示错误
>                     // TODO
>                 }
>             }
>         }
>     }
>     shape = {b: shapes[0][0], c: shapes[0][1], w: shape[0][2], h: shape[0][3]}
> ```

-----------

#### 实例

![](/notebooks/assets/layers/cat_layer.png)

### ~~Loss层~~

**删除或者隐藏**

# <div align='center'> 拖拽模型 </div>

In [2]:
from k12libs.utils.nb_easy import k12ai_start_html, W3URL
k12ai_start_html(f'{W3URL}/drawnet.html?jfile=simple', height=780, flask=True)

 * Serving Flask app "k12libs.utils.nb_easy" (lazy loading)


 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## Input/Output Layers

###  Input

### Output

## Core Layers

###  Activation

###  ActivityRegularization

### Dense

### Dropout

### Flatten

### Highway

### Lambda

### Masking

### MaxoutDense

### Permute

### RepeatVector

### Reshape

### SpatialDropout1D

### SpatialDropout2D

### SpatialDropout3D

### TimeDistributedDense

## Convolutional Layers

### AtrousConvolution1D

### AtrousConvolution2D

### Convolution1D

### Convolution2D

### Convolution3D

### Cropping1D

### Cropping2D

### Cropping3D

### Deconvolution2D

### SeparableConvolution2D

### UpSampling1D

### UpSampling2D

### UpSampling3D

### ZeroPadding1D

### ZeroPadding2D

### ZeroPadding3D

## Pooling Layers

### AvgPooling1D

### AvgPooling2D

### AvgPooling3D

### MaxPooling1D

### MaxPooling2D

### MaxPooling3D

### GlobalAvgPooling1D

### GlobalAvgPooling2D

### GlobalAvgPooling3D

### GlobalMaxPooling1D

### GlobalMaxPooling2D

### GlobalMaxPooling3D

## Local Layers

### LocallyConnected1D

### LocallyConnected2D

## Recurrent Layers

### GRU

### LSTM

### SimpleRNN

## Embeddings Layers

### Embeding

## Convolutional Recurrent Layers

### ConvLSTM2D

### ConvRecurrent2D

## Advanced Activations Layers

### ELU

### LeakyReLU

### ParametricSoftplus

### PReLU

### SReLU

### ThreasholdedReLU

## Normalization Layers

### BatchNormalization

## Noise Layers

### GaussianDropout

### GaussianNoise

## Special Functions

### Merge

## Pre-Trained Models

### InceptionV3

### ResNet50

### SqueezeNet

### VGG16

### VGG19

### WideResNet

## References

### [deepcognition](https://app.deepcognition.ai)